CODE FOR EMOTION CLASSIFIER AND EMOJI ADDITION


1. DATA PRE-PROCESSING FOR TRAINING EMOTION CLASSIFIER


In [1]:
import nltk
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict

from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/Shared drives/NLP/Project_Files/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'

import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


#THIS IS THE TRAIN CSV FILE 

train = pd.read_csv(datapath + 'train.csv')
# data = data.dropna()
train[:10]

#THIS IS THE TEST CSV FILE 

test = pd.read_csv(datapath + 'test.csv')
test[:10]

# CHECK DIFFERENT KEYS IN THE CSV
test.keys(), train.keys()

# PRE-PROCESSING OF TRAIN and TEST DATA

del train['conv_id'], train['utterance_idx'], train['speaker_idx'], train['selfeval'], train['tags']
del train['Unnamed: 8'], train['Unnamed: 9'], train['Unnamed: 10']
train = train.drop_duplicates(subset=('prompt'), keep='first', inplace=False)
del train['utterance']
del test['conv_id'], test['utterance_idx'], test['speaker_idx'], test['selfeval'], test['tags']
test = test.drop_duplicates(subset=('prompt'), keep='first', inplace=False)
del test['utterance']


# NOW FIND UNIQUE EMOTIONS IN THE DATASET
train_unique = []
for i in range(len(train)):
    if train.iloc[i]['context'] not in train_unique:
        train_unique.append(train.iloc[i]['context'])
print(train_unique)
print(len(train_unique))

test_unique = []
for i in range(len(train)):
    if train.iloc[i]['context'] not in test_unique:
        test_unique.append(train.iloc[i]['context'])
print(test_unique)
print(len(test_unique))
train = train.dropna()
test=test.dropna()
train[:10]
test[:10]

#STOP WORD PROCESSING, CURRENTLY SET TO FALSE

stop_words_en = stopwords.words('english')
def cleanText(raw_text, stopwords=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'html').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case t

    if stopwords:
        words = [i for i in words if i not in stop_words_en]
    
    return( str(" ".join(words)))

cleanTextvec = np.vectorize(cleanText)    
filtered_train = train.copy()
filtered_test = test.copy()

train['prompt'] = cleanTextvec(train['prompt'], False)
test['prompt'] = cleanTextvec(test['prompt'], False)


# MAKE EMOTION DICTIONARY
train['Rating'] = 0
test['Rating'] = 0
emodict = {'amusement':0, 'satisfaction':1, 'optimism':2, 'pride in achievement':3, 'contentment':4,
           'anger':5, 'fear':6, 'disgust':7, 'sadness':8, 'contempt':9}
#for i in test_unique:
#    print(i, emodict[i])
for emotion in emodict:
    train['Rating'][train['context']==emotion] = emodict[emotion]
    test['Rating'][test['context']==emotion] = emodict[emotion]

print(train[train['context'] == train_unique[10]])
print(train[train['context'] == train_unique[12]])
print(train[train['context'] == train_unique[13]])
print(train[train['context'] == train_unique[14]])
train = train.drop([4811,6392,7571,61943], axis=0)

# ALL PRE-PROCESSING DONE, NOW SAVE IT TO CSV FILE

train.to_csv(datapath + 'amit_train_emotion.csv',index=False)# FALSE so that INDEX IS NOT SAVED
test.to_csv(datapath + 'amit_test_emotion.csv', index=False)
train[:10]
test[:10]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['sadness', 'fear', 'pride in achievement', 'optimism', 'contentment', 'anger', 'contempt', 'satisfaction', 'disgust', 'amusement', "oh no. o'm sorry to here that.", nan, 'Did you call the police?', '296', "I don't blame you. I don't know anything about Horses_comma_ but congrats on winning the bet"]
15
['sadness', 'fear', 'pride in achievement', 'optimism', 'contentment', 'anger', 'contempt', 'satisfaction', 'disgust', 'amusement', "oh no. o'm sorry to here that.", nan, 'Did you call the police?', '296', "I don't blame you. I don't know anything about Horses_comma_ but congrats on winning the bet"]
15


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                             context prompt  Rating
4811  oh no. o'm sorry to here that.              0
                       context prompt  Rating
6392  Did you call the police?              0
     context                                             prompt  Rating
7571     296  yes people and pets too soon so it definitely ...       0
                                                 context prompt  Rating
61943  I don't blame you. I don't know anything about...              0


,context,prompt,Rating
0,disgust,i felt guilty when i was driving home one nigh...,7
5,contentment,my mother stopped by my house one day and said...,4
9,contempt,i just broke up with my girlfriend comma we we...,9
13,amusement,i received concert tickets for christmas,0
17,sadness,i ve read an article about a little newborn ba...,8
22,contentment,my friend s baby hurt himself yesterday i was ...,4
27,optimism,i really hope my husband finds a full time job...,2
32,anger,some guys shot my neighbour and ran into the w...,5
36,contentment,i m going to see my parents soon,4
40,anger,i felt very angry when a co worker of mine pre...,5


2. LOAD PRE-PROCESS DATASETs

In [3]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict

from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/Shared drives/NLP/Project_Files/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'

import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

# LOAD CSV FILES
train = pd.read_csv(datapath + 'amit_train_emotion.csv')
test = pd.read_csv(datapath + 'amit_test_emotion.csv')
train = train.dropna()
test=test.dropna()
#train.set_index('Unnamed: 0')
#test.set_index('Unnamed: 0')

# NOW MAKE IT READY FOR TRAINING
del train['context'], test['context']
# train, test = train_test_split(data, test_size = 0.2, random_state = 42)
train, val = train_test_split(train, test_size = 0.1, random_state = 42)
y_train = np.array(train['Rating'])
y_val = np.array(val['Rating'])
y_test = np.array(test['Rating'])

# CHECK NUMBER OF TRAIN, VALIDATION AND TEST 
print(train.shape)
print(val.shape)
print(test.shape)

max_features = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
maxlen = 1000
batch_size = 128
nb_classes = 10


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(16317, 2)
(1814, 2)
(2492, 2)


In [0]:
train = pd.read_csv(datapath + 'amit_train_emotion.csv')
test = pd.read_csv(datapath + 'amit_test_emotion.csv')
train = train.dropna()
test=test.dropna()

val=pd.read_csv(datapath + 'akhilesh_val_emotion.csv')
y_train = np.array(train['Rating'])
y_val = np.array(val['Rating'])
y_test = np.array(test['Rating'])
print(train.shape)
print(val.shape)
print(test.shape)

max_features = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
maxlen = 1000
batch_size = 128
nb_classes = 10

(18131, 3)
(1814, 2)
(2492, 3)


3. FITTING TOKENIZER


In [0]:

tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(test['prompt'])
f = open(datapath + 'amit_normal_tokenizer_prompts.dat', 'wb')
p.dump(tokenizer, f)
f.close()


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


4. LOAD TOKENIZER


In [4]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict

from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/Shared drives/NLP/Project_Files/'
modelpath = '/content/gdrive/Shared drives/NLP/Project_Files/Models/EmotionClassifier'

import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

f = open(datapath + 'amit_normal_tokenizer_prompts.dat', 'rb')
tokenizer=p.load(f)
f.close()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


5. TOKENIZATION 


In [5]:
# vectorize the text samples into a 2D integer tensor

sequences_train = tokenizer.texts_to_sequences(train['prompt'])
sequences_val = tokenizer.texts_to_sequences(val['prompt'])
sequences_test = tokenizer.texts_to_sequences(test['prompt'])


print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_val = sequence.pad_sequences(sequences_val, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)


Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_train shape:', X_val.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (16317, 1000)
X_train shape: (1814, 1000)
X_test shape: (2492, 1000)


CNN Model Defination


In [6]:
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def max_1d(X):
    return K.max(X, axis=1)

from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.models import Model
from keras.layers import Input,Flatten, Dense, Embedding, RNN, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, concatenate, Lambda
from keras import optimizers


nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


# nb_filter *= 2

print('Build model...')
cmodel1 = Sequential()
cmodel1.add(Embedding(max_features, 1000))
cmodel1.add(SpatialDropout1D(0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
cmodel1.add(Convolution1D(nb_filter=nb_filter,
                       filter_length=filter_length,
                       border_mode='valid',
                       activation='tanh',
                       subsample_length=1))

#cmodel1.add(BatchNormalization())
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)

cmodel1.add(Lambda(max_1d, output_shape=(nb_filter,)))
cmodel1.add(Dense(hidden_dims))
cmodel1.add(Dropout(0.2))
cmodel1.add(Activation('relu'))
cmodel1.add(Dense(nb_classes))
cmodel1.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
cmodel1.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])


cmodel1.summary()

Build model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 1000)        20000000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 1000)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 300)         900300    
_________________________________________________________________
lambda_1 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
______________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`


CNN MODEL TRAINING


In [7]:
print('Train...')
cmodel1.fit(X_train, Y_train, batch_size=batch_size, epochs=2,
         validation_data=(X_val, Y_val))
score, acc = cmodel1.evaluate(X_test, Y_test,
                           batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
from keras.models import model_from_json
model_json = cmodel1.to_json()
with open(datapath+"AMIT_epoch2W.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cmodel1.save_weights(datapath+"AMIT_epoch2W.h5")
print("Saved model to disk")
 
 

Train...



Train on 16317 samples, validate on 1814 samples
Epoch 1/2





16317/16317 [==============================] - 74s 5ms/step - loss: 0.2974 - acc: 0.9034 - val_loss: 0.2095 - val_acc: 0.9262
Epoch 2/2
2492/2492 [==============================] - 3s 1ms/step
Test score: 0.19212143517803418
Test accuracy: 0.9312600179621725
Saved model to disk


LOAD MODEL

In [10]:
# later...
 from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"AMIT_epoch2W.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(datapath+"AMIT_epoch2W.h5")
print("Loaded model from disk")
 





Loaded model from disk


MICRO ACCURACY


In [11]:
val_preds = loaded_model.predict_classes(X_val, verbose=0)
preds = loaded_model.predict_classes(X_test, verbose=0)
print('Validation micro accuracy: ', accuracy_score(val['Rating'], val_preds) * 100)
print('Test micro accuracy: ', accuracy_score(test['Rating'], preds) * 100)

Validation micro accuracy:  61.852260198456456
Test micro accuracy:  60.63402889245586


MACRO ACCURACY

In [12]:
s = 0
for i in range(10):
    s += accuracy_score(val[val['Rating']==i]['Rating'], val_preds[val['Rating']==i])
print('Validation macro accuracy:',s * 10)

s = 0
for i in range(10):
    s += accuracy_score(test[test['Rating']==i]['Rating'], preds[test['Rating']==i])
print('Test macro accuracy:',s * 10)

Validation macro accuracy: 59.91902290550014
Test macro accuracy: 58.26756187664914
